# Marching Squares
Takes a scalar field such as the Signed Distance Function (SDF) and generates contours consisting of line segment. A similar algorithm exists in 3D where a scalar field is turned into surfaces consisting of triangles.

In [ ]:
import numpy as np

In [ ]:
import seaborn as sns
sns.set_theme()
sns.set(style='darkgrid', context='talk', palette='Pastel1')
sns.set(style='dark')  # get rid of gridlines

In [ ]:
# configure matlab for notebooks
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 5]

In [ ]:
# load points cloud from the field interpolation
sdf = np.load('sdf-512-16.00.npy')

In [ ]:
plt.imshow(sdf); plt.colorbar()

In [ ]:
plt.contour(sdf, levels=[0]); plt.axis('equal')

## Algorithm
* Each 2x2 block of the SDF is iterated
* Each block is thresholded against the level (e.g. 0) as either below or not below.
* There are 16 (2^4) different configurations each yielding a specific set of lines (or no line) cutting the square
* The exact vertex positions of the lines are computed by lineary interpolating the positions

In [ ]:
# TODO: Zoom in on single 2x2 block

In [ ]:
# 0 -- 1
# |    |
# 2 -- 3

LOOKUP = [
    (),
    (((0, 2), (0, 1)),),
    (((0, 1), (1, 3)),),
    (((0, 2), (1, 3)),),
    (((0, 2), (2, 3)),),
    (((0, 1), (2, 3)),),
    (((0, 2), (2, 3)), ((0, 1), (1, 3))),
    (((2, 3), (1, 3)),),
    (((1, 3), (2, 3)),),
    (((0, 2), (0, 1)), ((2, 3), (1, 3))),
    (((2, 3), (0, 1)),),
    (((2, 3), (0, 2)),),
    (((1, 3), (0, 2)),),
    (((1, 3), (0, 1)),),
    (((0, 1), (0, 2)),),
    (),    
]


In [ ]:
def average_edge(points: np.ndarray, edge: Edge) -> np.ndarray:
    i0, i1 = edge
    return (points[i0] + points[i1]) * 0.5

In [ ]:
def to_bits(number: int, n: int) -> np.ndarray:
    return np.array([bool(number & (1 << i)) for i in range(n)], dtype=bool)

POINTS = np.array([
    (0, 0),
    (1, 0),
    (0, 1),
    (1, 1),
])
fig, axes = plt.subplots(4, 4)
plt.subplots_adjust(wspace=0.5, hspace=0.8)

for i in range(16):
    row, column = i // 4, i % 4
    plot = axes[row, column]
    plot.set_title(f'case {i}')
    plot.axis('off'); plot.axis('equal'); plot.axis([0, 1, 1, 0])

    bits = to_bits(i, n=4)
    plot.scatter(POINTS[bits, 0], POINTS[bits, 1], facecolors='r', edgecolors='r')
    plot.scatter(POINTS[~bits, 0], POINTS[~bits, 1], facecolors='none', edgecolors='r')
    for e0, e1 in LOOKUP[i]:
        a, b = average_edge(POINTS, e0), average_edge(POINTS, e1)
        plot.plot([a[0], b[0]], [a[1], b[1]], color='blue')

In [ ]:
from typing import Tuple


def marching_squares_nolerp(sdf: np.ndarray, level: float = .0) -> Tuple[np.ndarray, np.ndarray]:
    h, w = sdf.shape
    vertices = []
    lines = []
    for y in range(h - 1):
        for x in range(w - 1):
            values = np.array([
                sdf[y, x],
                sdf[y, x + 1],
                sdf[y + 1, x],
                sdf[y + 1, x + 1],
            ])
            block = values < level
            index = sum(int(bit) * 1 << i for i, bit in enumerate(block.ravel()))
            p = np.array([x, y])
            for e0, e1 in LOOKUP[index]:
                a, b = average_edge(POINTS, e0), average_edge(POINTS, e1)
                lines.append((len(vertices), len(vertices) + 1))
                vertices.extend([p + a, p + b])

    return np.vstack(vertices), lines

vertices, lines = marching_squares_nolerp(sdf)

In [ ]:
from PIL import Image
from PIL import ImageDraw

def draw_lines(vertices: np.ndarray, lines: List[Tuple[int, int]], linewidth: int = 2) -> None:
    im = Image.new('RGB', (1024, 512), (0xff, 0xff, 0xff))
    draw = ImageDraw.Draw(im)
    color = (0x00, 0x00, 0x00)
    scale = 16
    for a, b in lines:
        draw.line((tuple(scale * vertices[a]), tuple(scale * vertices[b])), fill=color, width=linewidth)
    return im

draw_lines(vertices, lines)

In [ ]:
def interpolate_edge(points: np.ndarray, values: np.ndarray, edge: Edge, level: float) -> np.ndarray:
    i0, i1 = edge
    v0, v1 = values[i0], values[i1]
    # make sure v0 is smaller that v1
    if v0 > v1:
        i0, i1 = i1, i0
        v0, v1 = v1, v0

    # compute parameter
    t = (level - v0) / (v1 - v0)
    # lerp
    return (1 - t) * points[i0] + t * points[i1]


def marching_squares(sdf: np.ndarray, level: float = .0) -> Tuple[np.ndarray, np.ndarray]:
    h, w = sdf.shape
    vertices = []
    lines = []
    for y in range(h - 1):
        for x in range(w - 1):
            values = np.array([
                sdf[y, x],
                sdf[y, x + 1],
                sdf[y + 1, x],
                sdf[y + 1, x + 1],
            ])
            block = values < level
            index = sum(int(bit) * 1 << i for i, bit in enumerate(block.ravel()))
            p = np.array([x, y])
            for e0, e1 in LOOKUP[index]:
                a, b = interpolate_edge(POINTS, values, e0, level), interpolate_edge(POINTS, values, e1, level)
                lines.append((len(vertices), len(vertices) + 1))
                vertices.extend([p + a, p + b])

    return np.vstack(vertices), lines


vertices, lines = marching_squares(sdf)
draw_lines(vertices, lines)